# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [87]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import json
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [88]:
cities_df = pd.read_csv('../output_data/cities.csv')
cities_df.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [89]:
gmaps.configure(api_key=g_key)

In [90]:
# Store latitude and longitude in locations
lat_lon = cities_df[['Lat', 'Lng']]

# Fill NaN values and convert to float
humid = cities_df["Humidity"]
#lat_lon.head()

In [91]:
# Plot Heatmap
figure_layout = {
    'width': '900px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

# Use the gmaps.figure passing a zoom_level of 2 and a center point so the map displays properly at
# a readable size
fig = gmaps.figure(layout=figure_layout,zoom_level=2,center=(15,25))
# Create heat layer
heat_layer = gmaps.heatmap_layer(lat_lon, weights=humid, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1.5)

# Add heat layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [94]:
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 15 mph.
# Zero cloudiness.
crit_temperature = (cities_df['Max Temp'] > 70) & (cities_df['Max Temp'] < 80) 
crit_windSpeed = (cities_df['Wind Speed'] < 15) 
crit_cloudiness = (cities_df['Cloudiness'] == 0)
final_criteria = crit_temperature & crit_windSpeed & crit_cloudiness

# Use boolean indexing to filter the weather_df dataframe
ideal_weather_df = cities_df[final_criteria]
ideal_weather_df = ideal_weather_df.dropna()
ideal_weather_df = ideal_weather_df.reset_index()
ideal_weather_df.head(10)

,index,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,56,56,pisco,0,PE,1558378764,30,-13.71,-76.20,75.99,11.41
1,70,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32
2,88,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50
3,114,114,koumac,0,NC,1558378775,68,-20.56,164.28,74.40,12.86
4,138,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06
5,139,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59
6,176,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93
7,297,297,mogok,0,MM,1558378844,51,22.92,96.51,72.24,2.77
8,444,444,birjand,0,IR,1558378880,26,32.86,59.22,71.60,9.17


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [97]:
hotel_df = ideal_weather_df
# Add column for Hotel Name - Note that we used "" to specify initial entry.
hotel_df['Hotel Name'] = ""
#hotel_df.head()

In [101]:
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "keyword": "Hotel",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    # get lat, lng from hotel_df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
#     print the name_address url, avoid doing for public github repos in order to avoid exposing key
    #print(name_address.url)

    # convert to json
    name_address = name_address.json()
    #print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
        #hotel_df.loc[index, "Airport Address"] = name_address["results"][0]["vicinity"]
        #hotel_df.loc[index, "Airport Rating"] = name_address["results"][0]["rating"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
#hotel_df.info()

Missing field/result... skipping.


In [102]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [103]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer([(-34.42, 19.24),(32.68, -115.50),(19.71,-155.08)])
# Display figure
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…